In [264]:

"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding more
average features and weekly average features on it.
"""
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
import sys

import math
import sklearn.metrics as skl_metrics

from datetime import timedelta
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger



#------------------------------------------------------------------------------------#

df_train = pd.read_csv(
    '../input/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    parse_dates=["date"],    
)

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(['id'])



/home/zyp/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [265]:
df_T007 = pd.read_csv(
    "../submit/T007_MoreWKs_TrainToAug15.csv", 
).set_index(['id'])

In [266]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3370464 entries, 125497040 to 128867503
Data columns (total 4 columns):
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
onpromotion    bool
dtypes: bool(1), datetime64[ns](1), int64(2)
memory usage: 106.1 MB


In [267]:
df_fcst = pd.concat([df_test,df_T007], axis = 1, join = 'inner')
 

In [268]:
df_fcst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3370464 entries, 125497040 to 128867503
Data columns (total 5 columns):
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
onpromotion    bool
unit_sales     float64
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 131.8 MB


In [376]:

df_min_date = df_train[['item_nbr','store_nbr','date']].groupby(['item_nbr','store_nbr'])\
    ['date'].min().to_frame('min_date')
df_min_date.reset_index(inplace=True)
df_min_date_recent  = df_min_date.loc[(df_min_date.min_date >= '2017-08-09')]


df_sales_recent = pd.merge(df_min_date_recent, df_train, how='inner', on=['item_nbr','store_nbr'])\
    .groupby(['item_nbr','store_nbr'])['unit_sales'].sum().to_frame('sum_sales')
df_sales_recent.reset_index(inplace=True)

df_sales_recent.sum_sales = df_sales_recent.sum_sales / 7



In [377]:
df_sales_recent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 3 columns):
item_nbr     321 non-null int64
store_nbr    321 non-null int64
sum_sales    321 non-null float64
dtypes: float64(1), int64(2)
memory usage: 7.6 KB


In [378]:
df_new_items = df_sales_recent[['item_nbr','store_nbr','sum_sales']].groupby(['item_nbr'])\
    ['sum_sales'].mean().to_frame('avg_item_sales')
df_new_items.reset_index(inplace=True)

In [380]:
df_new_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 2 columns):
item_nbr          89 non-null int64
avg_item_sales    89 non-null float64
dtypes: float64(1), int64(1)
memory usage: 1.5 KB


In [381]:
df_fcst_new = pd.merge(df_fcst,df_new_items,how = 'left',on=['item_nbr'])

In [382]:
df_fcst_new.head()

,id,date,store_nbr,item_nbr,onpromotion,unit_sales,avg_item_sales
0,125497040,2017-08-16,1,96995,False,0.2107,NaN
1,125497041,2017-08-16,1,99197,False,0.4035,NaN
2,125497042,2017-08-16,1,103501,False,0.0000,NaN
3,125497043,2017-08-16,1,103520,False,1.3555,NaN
4,125497044,2017-08-16,1,103665,False,2.0158,NaN


In [384]:
df_fcst_new.loc[(df_fcst_new['unit_sales'] == 0) & ( df_fcst_new['avg_item_sales'] > 0),].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39763 entries, 1951 to 3370427
Data columns (total 7 columns):
id                39763 non-null int64
date              39763 non-null datetime64[ns]
store_nbr         39763 non-null int64
item_nbr          39763 non-null int64
onpromotion       39763 non-null bool
unit_sales        39763 non-null float64
avg_item_sales    39763 non-null float64
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3)
memory usage: 2.2 MB


In [370]:
pos_idx = (df_fcst_new['unit_sales'] == 0) & ( df_fcst_new['avg_item_sales'] > 0)
df_fcst_new_pos = df_fcst_new.loc[pos_idx]
 

In [373]:
df_fcst_new.loc[pos_idx].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48522 entries, 178 to 3370427
Data columns (total 7 columns):
id                48522 non-null int64
date              48522 non-null datetime64[ns]
store_nbr         48522 non-null int64
item_nbr          48522 non-null int64
onpromotion       48522 non-null bool
unit_sales        48522 non-null float64
avg_item_sales    48522 non-null float64
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3)
memory usage: 2.6 MB


In [350]:
df_fcst_new.loc[pos_idx,'unit_sales'] = df_fcst_new_pos['avg_item_sales']

In [328]:
df_new_items.reset_index(inplace=True)
df_fcst_new.head()

,date,store_nbr,item_nbr,onpromotion,unit_sales,avg_item_sales
0,2017-08-16,1,96995,False,0.2107,NaN
1,2017-08-16,1,99197,False,0.4035,NaN
2,2017-08-16,1,103501,False,0.0000,NaN
3,2017-08-16,1,103520,False,1.3555,NaN
4,2017-08-16,1,103665,False,2.0158,NaN


In [336]:

pos_idx = (df_fcst_new['unit_sales'] == 0) & ( df_fcst_new['avg_item_sales'] > 0),

In [330]:
df_fcst.reset_index(inplace=True)

In [355]:
print('sum of old forecast', df_fcst.unit_sales.sum())

sum of old forecast 9946141.877598988


In [356]:
print('sum of new forecast', df_fcst_new.unit_sales.sum())

sum of new forecast 9983395.804764258


In [360]:
df_fcst_new[['id', 'unit_sales']].to_csv('T100_NewSKU_V01.csv.gz', index=False, float_format='%.3f', compression='gzip')